## Introduction
At this point, the number of optional parameters for each submodule (`MavTracer`, `MavLayout`, etc) was growing and seemed likely to continue growing, so I wished to simplify the main library interface, allowing the user to specify as few or as many parameters as desired, while keeping the code base easy to maintain and the parameters easy to document.

The following criteria were deemed desirable:
* Backwards compatibility: it should be possible to specify any parameter as a kwarg on either the `MAV` or submodule interface
* All optional parameters should have suitable default values
* Little or no code duplication for maintainability purposes
* Intellisense: parameter type hints and doc strings should be parsable in VSCode

## Experiments

### Defining the dataclass

In [ ]:
from dataclasses import dataclass
from typing import Tuple

@dataclass
class MavOptions:
    """
    param1 specifies which thing must be used to achieve some goal

    param2 determines which branch of this other algorithm to use for some use case 

    param3 specifies two things: some thing and some other thing
    """
    param1: int = 1    
    param2: str = '2'
    param3: Tuple[bool,str] = (False,'no')

opts = MavOptions(param2='two')
opts

MavOptions(param1=1, param2='two', param3=(False, 'no'))

In [10]:
opts.param1 = 111
opts

MavOptions(param1=111, param2='two', param3=(False, 'no'))

In [16]:
opts.unknown_param = 4
opts

MavOptions(param1=111, param2='two', param3=(False, 'no'))

In [27]:
opts.__setattr__('param1',1234)
opts

MavOptions(param1=1234, param2='two', param3=(False, 'no'))

### Passing the dataclass with kwargs to a submodule

In [28]:
class MavSubmodule:
    def __init__(self, opts:MavOptions=MavOptions(), **kwargs):
        self.opts = opts
        for k,v in kwargs.items():
            self.opts.__setattr__(k,v)
    
    def __repr__(self):
        return self.opts.__repr__()
    
MavSubmodule()

MavOptions(param1=1, param2='2', param3=(False, 'no'))

In [29]:
MavSubmodule(opts=MavOptions(param1='yes', param2='no', param3='maybe'))

MavOptions(param1='yes', param2='no', param3='maybe')

In [30]:
MavSubmodule(param1='yes', param2='no', param3='maybe')

MavOptions(param1='yes', param2='no', param3='maybe')

In [ ]:
MavSubmodule(opts=MavOptions(param1='yes', param2='no'), param3='maybe')

MavOptions(param1='yes', param2='no', param3='maybe')

## Observations
* Dataclasses will achieve the aims mentioned above
* Parameters passed to a submodule within a `MavOptions` object will enjoy the full intellisense experience
* Parameters passed as kwargs directly to the submodule `__init__` will have the same effect (and in fact override those passed within a `MavOptions` object), but without the intellisense
* The same `MavOptions` object can be passed to all submodules and each submodule can use only the desired fields
* Parameters can all be documented in one place, namely in the docstring of `MavOptions` 
* Dataclasses provide a neat way to specify type hints and default values